<a href="https://colab.research.google.com/github/SSolanoRuniandes/Notebooks-Aprendizaje-por-Refuerzo-Profundo/blob/main/TareaSemana3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![MAIA banner](https://raw.githubusercontent.com/MAIA4361-Aprendizaje-refuerzo-profundo/Notebooks_Tareas/main/Images/Aprendizaje_refuerzo_profundo_Banner_V1.png)

# <h1><center>Tarea Tutorial - Semana 3 <a href="https://colab.research.google.com/github/SSolanoRuniandes/Notebooks-Aprendizaje-por-Refuerzo-Profundo/blob/main/TareaSemana3.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" width="140" align="center"/></a></center></h1>

<center><h1>Deep Q Networks</h1></center>

En este tutorial...


# Tabla de Contenidos
1. [Objetivos de Aprendizaje](#scrollTo=Objetivos_de_Aprendizaje)  
2. [Marco Teórico](#scrollTo=Marco_Te_rico)  
3. [Instalación de Librerías](#scrollTo=Instalaci_n_de_Librer_as)  
4. [Familiarización con el Entorno de Gym](#scrollTo=Familiarizaci_n_con_el_Entorno_de_Gym)  
5. [DQN](#scrollTo=DQN)
6. [Doble DQN](#scrollTo=Doble_DQN)  
7. [Experience Replay](#scrollTo=Experience_Replay)  
8. [Reflexiones Finales](#scrollTo=Reflexiones_Finales)  
9. [Referencias](#scrollTo=Referencias)

# Objetivos de Aprendizaje  
  



# Marco Teórico  



# Instalación de Librerías  

...


In [1]:
#Descarga librerías no incluidas en Colab usando pip
!pip install stable_baselines3 #Stable Baselines3 -> Framework de Reinforcement Learning
!pip install sb3-contrib #SB3-Contrib es un repositorio aparte con otros algoritmos
!pip install ale-py #ALE se utiliza para el ambiente de Atari
!pip install "gymnasium[atari,accept-rom-license]" stable-baselines3 autorom renderlab -q #Gymnasium, envs de Atari y ROM
!AutoROM --accept-license
!pip install renderlab #usado para renderizar gym

#Importa estas librerías
import stable_baselines3 #importa Stable Baselines3
from stable_baselines3 import DQN #importa el agente/algoritmo de DQN
from stable_baselines3.common.logger import configure #importa herramientas de logger/debug
from stable_baselines3.common.logger import Logger, CSVOutputFormat, HumanOutputFormat #importa herramientas de logger/debug
from stable_baselines3.common.evaluation import evaluate_policy #importa herramienta de evaluación automática
from sb3_contrib import QRDQN #importa el agente/algoritmo de QRDQN
import gymnasium #importa la libreria de gymnasium con las simulaciones
import renderlab #importa renderlab para los videos
import ale_py #importa ale para los ambientes de Atari
from gymnasium.wrappers import TimeLimit #importa timelimit para acortar los episodios

#Importa otras librerías básicas
import numpy as np
import matplotlib.pyplot as plt
import random
import math
import pandas as pd
import sys

#Limpia los registros generados
from IPython.display import clear_output
clear_output()
print("Todas las librerías han sido instaladas correctamente.")

Todas las librerías han sido instaladas correctamente.


# Familiarización con el Entorno de Gym

In [2]:
env_render = gymnasium.make("ALE/Freeway-v5", render_mode="rgb_array") #Se crea el ambiente. Para este tutorial, utilice gymnasium si va a renderizar.
env_render = renderlab.RenderFrame(env_render, "./output") #Se crea una copia que se pueda renderizar con renderlab

terminated = False #Inicializa una condición para el loop
truncated = False #Inicializa una condición para el loop
total_reward=0 #Inicializa contador del retorno

obs , info = env_render.reset() #Se reinicia el estado para comenzar. En obs se almacena el estado observado (continuo, 2 dimensiones)

while not (terminated or truncated): #Simula hasta que el carro salga del valle o hasta que pasen 200 episodios
  action = 1
  #print(action)
  obs, reward, terminated, truncated, info = env_render.step(action)
  total_reward+=reward

print("Recompensa obtenida en el episodio:",total_reward) #Se imprime la recompensa obtenida
print("\n\n")

env_render.play() #Con esta función se obtiene el video de la simulación

Recompensa obtenida en el episodio: 21.0



Moviepy - Building video temp-{start}.mp4.
Moviepy - Writing video temp-{start}.mp4



Moviepy - Done !
Moviepy - video ready temp-{start}.mp4


In [ ]:
# Simulación

# =====================================================
# COMPLETAR ===========================================
#

# =====================================================

<OrderEnforcing<PassiveEnvChecker<AtariEnv<ALE/Freeway-v5>>>>


#DQN

# **DQN - Intento 1**
No hay pasos muertos al principio.

Sin nada, ni doble ni prrioritized experience replay

In [ ]:
from gymnasium.wrappers import TimeLimit

gymnasium.register_envs(ale_py) #Hay que registrar los entornos de ALE manualmente
env = gymnasium.make("ALE/Freeway-v5") #Crea el ambiente de Freeway



#Útil: Limitar el tiempo del episodio
env = TimeLimit(env, max_episode_steps=512)

#Importante: Limitar el buffer por las limitaciones de Colab
#Útil: learning_Starts para ganar algo de experiencia (8 episodios)
model = DQN("CnnPolicy", env, buffer_size=50_000, verbose=1, learning_starts=4096)

#200 episodios de entrenamiento
model.learn(total_timesteps=102_400, log_interval=5)
model.save("dqn_freeway")


Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 512      |
|    ep_rew_mean      | 0        |
|    exploration_rate | 0.762    |
| time/               |          |
|    episodes         | 5        |
|    fps              | 623      |
|    time_elapsed     | 4        |
|    total_timesteps  | 2560     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 512      |
|    ep_rew_mean      | 0        |
|    exploration_rate | 0.525    |
| time/               |          |
|    episodes         | 10       |
|    fps              | 456      |
|    time_elapsed     | 11       |
|    total_timesteps  | 5120     |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 1.89e-07 |
|    n_updates        | 

In [ ]:
model = DQN.load("dqn_freeway")


env_render = gymnasium.make("ALE/Freeway-v5", render_mode="rgb_array") #Se crea el ambiente. Para este tutorial, utilice gymnasium si va a renderizar.
env_render = renderlab.RenderFrame(env_render, "./output") #Se crea una copia que se pueda renderizar con renderlab

terminated = False #Inicializa una condición para el loop
truncated = False #Inicializa una condición para el loop
total_reward=0 #Inicializa contador del retorno

obs , info = env_render.reset() #Se reinicia el estado para comenzar. En obs se almacena el estado observado (continuo, 2 dimensiones)

while not (terminated or truncated): #Simula hasta que el carro salga del valle o hasta que pasen 200 episodios
  action, _states = model.predict(obs, deterministic=True)
  #print(action)
  obs, reward, terminated, truncated, info = env_render.step(action)
  total_reward+=reward

print("Recompensa obtenida en el episodio:",total_reward) #Se imprime la recompensa obtenida
print("\n\n")

env_render.play() #Con esta función se obtiene el video de la simulación

  warnings.warn(



Recompensa obtenida en el episodio: 21.0



Moviepy - Building video temp-{start}.mp4.
Moviepy - Writing video temp-{start}.mp4



Moviepy - Done !
Moviepy - video ready temp-{start}.mp4


# **DQN - Intento 2**
Hay pasos muertos al principio.

Sin nada, ni doble ni prrioritized experience replay

In [ ]:
from gymnasium import Wrapper

class DelayActionWrapper(Wrapper):
    def __init__(self, env, delay_steps=100, no_op_action=0):
        super().__init__(env)
        self.delay_steps = delay_steps
        self.current_step = 0

    def reset(self, **kwargs):
        self.current_step = 0
        return self.env.reset(**kwargs)

    def step(self, action):
        if self.current_step < self.delay_steps:
            action = 0
        obs, reward, terminated, truncated, info = self.env.step(action)
        self.current_step += 1
        return obs, reward, terminated, truncated, info

In [ ]:
from gymnasium.wrappers import TimeLimit

gymnasium.register_envs(ale_py) #Hay que registrar los entornos de ALE manualmente
env = gymnasium.make("ALE/Freeway-v5") #Crea el ambiente de Freeway



#Útil: Limitar el tiempo del episodio
env = DelayActionWrapper(env, delay_steps=512, no_op_action=0)
env = TimeLimit(env, max_episode_steps=1024)

#Importante: Limitar el buffer por las limitaciones de Colab
#Útil: learning_Starts para ganar algo de experiencia (4 episodios)
model = DQN("CnnPolicy", env, buffer_size=50_000, verbose=1, learning_starts=4096)

#100 episodios de entrenamiento
model.learn(total_timesteps=102_400, log_interval=5)
model.save("dqn_freeway")

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 1.02e+03 |
|    ep_rew_mean      | 0        |
|    exploration_rate | 0.525    |
| time/               |          |
|    episodes         | 5        |
|    fps              | 458      |
|    time_elapsed     | 11       |
|    total_timesteps  | 5120     |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 7.07e-08 |
|    n_updates        | 255      |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 1.02e+03 |
|    ep_rew_mean      | 0        |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 10       |
|    fps              | 255      |
|    time_elapsed     | 40       |
|    total_timesteps  | 

In [ ]:
model = DQN.load("dqn_freeway")


env_render = gymnasium.make("ALE/Freeway-v5", render_mode="rgb_array") #Se crea el ambiente. Para este tutorial, utilice gymnasium si va a renderizar.
#env_render = DelayActionWrapper(env, delay_steps=512, no_op_action=0)
env_render = renderlab.RenderFrame(env_render, "./output") #Se crea una copia que se pueda renderizar con renderlab

terminated = False #Inicializa una condición para el loop
truncated = False #Inicializa una condición para el loop
total_reward=0 #Inicializa contador del retorno

obs , info = env_render.reset() #Se reinicia el estado para comenzar. En obs se almacena el estado observado (continuo, 2 dimensiones)

step=0
while not (terminated or truncated): #Simula hasta que el carro salga del valle o hasta que pasen 200 episodios
  action, _states = model.predict(obs, deterministic=True)
  if step < 512:
        action = 0

  obs, reward, terminated, truncated, info = env_render.step(action)
  total_reward+=reward
  step += 1

print("Recompensa obtenida en el episodio:",total_reward) #Se imprime la recompensa obtenida
print("\n\n")

env_render.play() #Con esta función se obtiene el video de la simulación


Recompensa obtenida en el episodio: 0.0



Moviepy - Building video temp-{start}.mp4.
Moviepy - Writing video temp-{start}.mp4



Moviepy - Done !
Moviepy - video ready temp-{start}.mp4


# **DQN - Intento 3**
No hay pasos muertos al principio.

Con doble, sin prrioritized experience replay

In [ ]:
gymnasium.register_envs(ale_py) #Hay que registrar los entornos de ALE manualmente
env = gymnasium.make("ALE/Freeway-v5") #Crea el ambiente de Freeway


#Útil: Limitar el tiempo del episodio
env = TimeLimit(env, max_episode_steps=512)

#Importante: Limitar el buffer por las limitaciones de Colab
#Útil: learning_Starts para ganar algo de experiencia (8 episodios)
#model = DQN("CnnPolicy", env, buffer_size=50_000, verbose=1, learning_starts=4096)

model = QRDQN(
    "CnnPolicy",
    env,
    verbose=1,
    buffer_size=50_000,           # experience replay size
    learning_starts=4096,         # evita que aprenda antes de tener experiencia
    train_freq=4,                  # cada 4 pasos entrena
    target_update_interval=2048,  # frecuencia para actualizar la red objetivo
)

#200 episodios de entrenamiento
model.learn(total_timesteps=102_400, log_interval=5)
model.save("qrdqn_freeway")

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 512      |
|    ep_rew_mean      | 0        |
|    exploration_rate | 0.01     |
| time/               |          |
|    episodes         | 5        |
|    fps              | 659      |
|    time_elapsed     | 3        |
|    total_timesteps  | 2560     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 512      |
|    ep_rew_mean      | 0.5      |
|    exploration_rate | 0.01     |
| time/               |          |
|    episodes         | 10       |
|    fps              | 403      |
|    time_elapsed     | 12       |
|    total_timesteps  | 5120     |
| train/              |          |
|    learning_rate    | 5e-05    |
|    loss             | 0.03     |
|    n_updates        | 

In [ ]:
model = QRDQN.load("qrdqn_freeway")


env_render = gymnasium.make("ALE/Freeway-v5", render_mode="rgb_array") #Se crea el ambiente. Para este tutorial, utilice gymnasium si va a renderizar.
env_render = renderlab.RenderFrame(env_render, "./output") #Se crea una copia que se pueda renderizar con renderlab

terminated = False #Inicializa una condición para el loop
truncated = False #Inicializa una condición para el loop
total_reward=0 #Inicializa contador del retorno

obs , info = env_render.reset() #Se reinicia el estado para comenzar. En obs se almacena el estado observado (continuo, 2 dimensiones)

while not (terminated or truncated): #Simula hasta que el carro salga del valle o hasta que pasen 200 episodios
  action, _states = model.predict(obs, deterministic=True)
  #print(action)
  obs, reward, terminated, truncated, info = env_render.step(action)
  total_reward+=reward

print("Recompensa obtenida en el episodio:",total_reward) #Se imprime la recompensa obtenida
print("\n\n")

env_render.play() #Con esta función se obtiene el video de la simulación

  warnings.warn(



Recompensa obtenida en el episodio: 19.0



Moviepy - Building video temp-{start}.mp4.
Moviepy - Writing video temp-{start}.mp4



Moviepy - Done !
Moviepy - video ready temp-{start}.mp4


# Doble DQN

# Experience Replay

# Reflexiones Finales




# Referencias

[1] Sutton, R. S. and Barto, A. G. (2018). Reinforcement Learning: An Introduction. The MIT Press, second edition.

[2] Mnih, V., Kavukcuoglu, K., Silver, D., Graves, A., Antonoglou, I., Wierstra, D., and Riedmiller, M. (2013). Playing atari with deep reinforcement learning. cite arxiv:1312.5602Comment: NIPS Deep Learning Workshop 2013.

[3] Gym Documentation, Cart Pole. `https://www.gymlibrary.dev/environments/classic_control/cart_pole/`

[4] Stable Baselines3 Documentation, DQN. `https://stable-baselines3.readthedocs.io/en/master/modules/dqn.html`